# Data Loading to BigQuery

This notebook covers the setup of taking all of the files that we've discovered around Price Transparency and loading them into a standard format. 

We first need to do one, and then we will do the rest. See https://github.com/pauldria/ncssm-2022-jterm-price-transparency#data for more.

In [1]:
from google.cloud import bigquery

import pandas as pd
import urllib
import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

ModuleNotFoundError: No module named 'google'

Let's look at the first one from the list, from Alamance. 

In [2]:
filepath = "https://www.conehealth.com/560529994_Alamance-Regional-Medical-Center-Inc_standardcharges.csv"

We have to decode it into `utf-8` since it was given to us as a binary file. Sigh. Try it without the `decode` portion to see what it's like otherwise.

In [3]:
with urllib.request.urlopen(filepath) as f:
    print(f.readline().decode("utf-8"))
    print(f.readline().decode("utf-8"))
    print(f.readline().decode("utf-8"))

﻿Alamance Regional Medical Center,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

Updated 09/28/2021,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,



In [4]:
date_obtained = datetime.datetime.now().strftime("%Y-%m-%d")
date_provided = "2021-09-28"
name_of_hospital = "Alamance Regional Medical Center"

When we [read the file](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html), we force all columns types to be `object` (i.e. string) and have the default NA value to be an empty string. This is to prevent headaches later. 

We simply do not know the contents of this data, so this is a safe default option and we will explicitly cast and form the data later when we know more.

In [5]:
df = pd.read_csv(filepath, skiprows = 3, dtype = str, na_values = "")
df.columns = [x.strip() for x in df.columns]

In [6]:
df.fillna("", inplace = True)
for c in df.columns:
    df[c] = df[c].map(str.strip)

In [7]:
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113385 entries, 0 to 113384
Data columns (total 115 columns):
 #    Column                                                                                 Dtype 
---   ------                                                                                 ----- 
 0    Common Billing Code (CPT/HCPCS/MS-DRG)                                                 object
 1    NDC (for medications/drugs)                                                            object
 2    Rev Code                                                                               object
 3    Procedure Description                                                                  object
 4    Gross Charge                                                                           object
 5    De-identified Min Inpatient Negotiated Rate Across All Payers                          object
 6    De-identified Max Inpatient Negotiated Rate Across All Payers                     

In [8]:
df.shape

(113385, 115)

In [9]:
df.head()

,Common Billing Code (CPT/HCPCS/MS-DRG),NDC (for medications/drugs),Rev Code,Procedure Description,Gross Charge,De-identified Min Inpatient Negotiated Rate Across All Payers,De-identified Max Inpatient Negotiated Rate Across All Payers,De-identified Min Outpatient Negotiated Rate Across All Payers,De-identified Max Outpatient Negotiated Rate Across All Payers,AETNA WHOLE HEALTH Negotiated Inpatient Rate,AETNA WHOLE HEALTH Negotiated Outpatient Rate,AETNA CAROLINA PERFERRED Negotiated Inpatient Rate,AETNA CAROLINA PREFERRED Negotiated Outpatient Rate,AETNA DESIGNATED PRODUCTS Negotiated Inpatient Rate,AETNA DESIGNATED PRODUCTS Negotiated Outpatient Rate,AETNA FIRST HEALTH Negotiated Inpatient Rate,AETNA FIRST HEALTH Negotiated Outpatient Rate,AETNA MEDICARE Negotiated Inpatient Rate,AETNA MEDICARE Negotiated Outpatient Rate,AMBETTER Negotiated Inpatient Rate,AMBETTER Negotiated Outpatient Rate,BEACON HEALTH VALUE OPTIONS Negotiated Inpatient Rate,BEACON HEALTH VALUE OPTIONS Negotiated Outpatient Rate,BEECH STREET NETWORK Negotiated Inpatient Rate,BEECH STREET NETWORK Negotiated Outpatient Rate,BLUE CROSS BLUE SHIELD (Excluding Market Place plans) Negotiated Inpatient Rate,BLUE CROSS BLUE SHIELD (Excluding Market Place Plans) Negotiated Outpatient Rate,BLUE CROSS BLUE SHIELD MEDICARE Negotiated Inpatient Rate,BLUE CROSS BLUE SHIELD MEDICARE Negotiated Outpatient Rate,BRIGHT HEALTH Negotiated Inpatient Rate,BRIGHT HEALTH Negotiated Outpatient Rate,CARDINAL INNOVATIONS Negotiated Inpatient Rate,CARDINAL INNOVATIONS Negotiated Outpatient Rate,CARE & CARE MEDICARE Negotiated Inpatient Rate,CARE & CARE MEDICARE Negotiated Outpatient Rate,CAROLINA BEHAVIORAL HEALTH Negotiated Inpatient Rate,CAROLINA BEHAVIORAL HEALTH Negotiated Outpatient Rate,CENTIVO (ALL PAYER PLANS) Negotiated Inpatient Rate,CENTIVO (ALL PAYER PLANS) Negotiated Outpatient Rate,CHOICE CARE (ALL PAYER PLANS) Negotiated Inpatient Rate,CHOICE CARE (ALL PAYER PLANS) Negotiated Outpatient Rate,CIGNA ALL PAYER PLANS Negotiated Inpatient Rate,CIGNA ALL PAYER PLANS Negotiated Outpatient Rate,CIGNA MEDICARE ADVANTAGE Negotiated Inpatient Rate,CIGNA MEDICARE ADVANTAGE Negotiated Outpatient Rate,COMPYSCH COPORATION Negotiated Inpatient Rate,COMPYSCH COPORATION Negotiated Outpatient Rate,FIDELIS Medicare Advantage Negotiated Inpatient Rate,FIDELIS Medicare Advantage Negotiated Outpatient Rate,GATEWAY HEALTH Negotiated Inpatient Rate,GATEWAY HEALTH Negotiated Outpatient Rate,HEALTHTEAM ADVANTAGE Negotiated Inpatient Rate,HEALTHTEAM ADVANTAGE Negotiated Outpatient Rate,HOSPICE -Authora CARE Negotiated Inpatient Rate,HOSPICE -Authora CARE Negotiated Outpatient Rate,HUMANA (ALL PAYER PLANS ) Negotiated Inpatient Rate,HUMANA (ALL PAYER PLANS ) Negotiated Outpatient Rate,HUMANA MEDICARE Negotiated Inpatient Rate,HUMANA MEDICARE Negotiated Outpatient Rate,LIBERTY ADVANTAGE MEDICARE Negotiated Inpatient Rate,LIBERTY ADVANTAGE MEDICARE Negotiated Outpatient Rate,LONGEVITY HEALTH Negotiated Inpatient Rate,LONGEVITY HEALTH Negotiated Outpatient Rate,MAGELLAN Negotiated Inpatient Rate,MAGELLAN Negotiated Outpatient Rate,MEDCOST NETWORK Negotiated Inpatient Rate,MEDCOST NETWORK Negotiated Outpatient Rate,MEDCOST ULTRA NETWORK Negotiated Inpatient Rate,MEDCOST ULTRA NETWORK Negotiated Outpatient Rate,MUTUAL OF OMAHA Special Risk Negotiated Inpatient Rate,MUTUAL OF OMAHA Special Risk Negotiated Outpatient Rate,NC DEPT OF PUBLIC SAFETYSTATE INMATES Negotiated Inpatient Rate,NC DEPT OF PUBLIC SAFETYSTATE INMATES Negotiated Outpatient Rate,NC MEDICAID PREPAID HEALTH PLAN HEALTHY BLUE Negotiated Inpatient Rate,NC MEDICAID PREPAID HEALTH PLAN HEALTHY BLUE Negotiated Outpatient Rat,NC MEDICAID PREPAID HEALTHPLAN UNITED HEALTHCARE COMMUNITY Negotiated Inpatient Rate,NC MEDICAID PREPAID HEALTHPLAN UNITED HEALTHCARE COMMUNITY Negotiated Outpatient Rate,NC MEDICAID PREPAID HEALTHPLAN WELLCARE Negotiated Inpatient Rate,NC MEDICAID PREPAID HEALTHPLAN WELLCARE Necotiated Outpatient Rate,NYSHIP EMPIRE Nego

In [10]:
fixed_columns = df.columns[0:4]
melted_columns = df.columns[4:]
df_transformed = df.melt(id_vars = fixed_columns, value_vars = melted_columns, var_name = "payer", value_name = "cost")

In [11]:
df_transformed.shape

(12585735, 6)

In [13]:
df_transformed.columns = ["code", "ndc", "rev_code", "description", "payer", "cost"]

In [14]:
df_transformed.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12585735 entries, 0 to 12585734
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   code         object
 1   ndc          object
 2   rev_code     object
 3   description  object
 4   payer        object
 5   cost         object
dtypes: object(6)
memory usage: 576.1+ MB


## Now we are ready to load to BigQuery